In [0]:

## Libraries Prequisities
!pip install scikit-learn
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 200kB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2


In [0]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0-rc2
/device:GPU:0
Num GPUs Available:  1


In [0]:
from tensorflow.keras.callbacks import Callback

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
np.random.seed(12227)

In [0]:
n_ep = 200
loss = 0.2
bs = 1000

In [0]:
def custom_stopping(value=0.5, verbose=0):
    early = EarlyStoppingByLossVal(monitor='val_loss', value=value, verbose=verbose)
    return early

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_acc', value=0.95, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        # if current is None:
        # warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [0]:
def DataPreparation(data_input_file):
  print('Kasnesis  et al. 2018 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  # X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  # classes_number = Y.shape[1]
  # Y = np.argmax(Y, axis=1)
  return X,Y,folds

In [0]:
X,Y,f=DataPreparation('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

Bevilacqua  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz


In [0]:
print(X[2].shape)

(1, 250, 23)


In [0]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [0]:
def RunKasnesis(data_input_file):
  X,Y,folds,= DataPreparation(data_input_file)
  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [0]:
def perception_net(input_dim, num_classes, filters=(48, 96, 96), dilation=False, separate_modalities=False):
    """ PerceptionNet Model.
    See paper: https://arxiv.org/abs/1811.00170
    Arguments:
        input_dim: 2D input array, where the first value is the number of signals
            and the second the time steps.
        num_classes: Number of classes to predicts.
        filters: Optional, amount of feature maps employed by the convolutions.
        dilation: Optional, true whether dilation should be employed in the
            1D Conv blocks.
    """

    if dilation:
        kernel_width = 8
        dilation_rate = 2
    else:
        kernel_width = 15
        dilation_rate = 1

    input_tensor = tf.keras.Input(shape=(1,input_dim[0], input_dim[1]))
    x = tf.keras.layers.Reshape(target_shape=(1,input_dim[0], input_dim[1]))(input_tensor)

    # First "Conv1D" block
    x = tf.keras.layers.Conv2D(filters=filters[0],
               kernel_size=(1,kernel_width), # height, width
               dilation_rate=dilation_rate,
               kernel_initializer='random_uniform',
               padding="same")(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(1,2),
                  strides=(1,2))(x)
    x = tf.keras.layers.Dropout(rate=0.4)(x)

    # Second "Conv1D" block
    x = tf.keras.layers.Conv2D(filters=filters[1],
               kernel_size=(1,kernel_width), #height, width
               dilation_rate=dilation_rate,
               kernel_initializer='random_uniform',
               padding="same")(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(1,2),
                  strides=(1,2))(x)
    x = tf.keras.layers.Dropout(rate=0.4)(x)

    # Late sensor fusion
    x = tf.keras.layers.Conv2D(filters=filters[2],
               kernel_size=(3,15),
               strides=(3,1),
               kernel_initializer='random_uniform',
               padding="same")(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(rate=0.4)(x)

    # Classification
    x = tf.keras.layers.Dense(units=num_classes, activation='softmax')(x)

    return tf.keras.Model(input_tensor, x, name='PerceptionNet')


In [0]:
# def build_model_1(row,col,num_classes):

 
 
#   layers = [
#         tf.keras.layers.Conv2D(filters=10,kernel_size=(3, 5), activation = tf.nn.relu),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.MaxPool2D(pool_size=(3,3),strides = 1),

#         tf.keras.layers.Conv2D(filters=2,kernel_size=(2, 4),activation = tf.nn.relu),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = 1),

#         tf.keras.layers.Conv2D(filters=2,kernel_size=(2, 2),activation = tf.nn.relu),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.MaxPool2D(pool_size=(3,2),strides = 1),
        
        

       

       
#         tf.keras.layers.Flatten(),
#         tf.keras.layers.Dense(500, activation = tf.nn.relu),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(25, activation = tf.nn.relu),
#         tf.keras.layers.Dropout(0.5),
       
#         tf.keras.layers.Dense(num_classes, activation = 'softmax')
#     ]
    
#   model = tf.keras.Sequential(layers)
#   return model

In [0]:
batch_size = 64
epochs = 2000
validation_split = 0.15
stopping_patience = 100
model_save_frequency = 50
# AdaDelta parameters
learning_rate = 1.0
rho = 0.95
epsilon = 1e-08


In [0]:
def Train(X,y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _, _,img_rows, img_cols = X.shape
  print(X.shape)
  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    model=perception_net(input_dim=(img_rows, img_cols), num_classes=n_class, filters=(12,24,24), dilation=False)
    optimizer = tf.keras.optimizers.Adadelta(lr=learning_rate,rho=rho, epsilon=epsilon)
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=stopping_patience, restore_best_weights=True),
        
    ]

    model.compile(loss="categorical_crossentropy",
                  optimizer=optimizer,
                  metrics=['accuracy'])
    # print(model.summary)
    
    model.fit(X_train, y[train_idx], batch_size=batch_size,epochs=epochs, validation_split=validation_split, shuffle=True,callbacks=callbacks)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1

In [0]:
tf.keras.backend.set_image_data_format('channels_first')

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz
(2555, 1, 250, 23)
Epoch 1/2000
31/31 [==============================] - 0s 15ms/step - loss: 2.3567 - accuracy: 0.1038 - val_loss: 2.5767 - val_accuracy: 0.0000e+00
Epoch 2/2000
31/31 [==============================] - 0s 11ms/step - loss: 2.2380 - accuracy: 0.1340 - val_loss: 2.6346 - val_accuracy: 0.0000e+00
Epoch 3/2000
31/31 [==============================] - 0s 11ms/step - loss: 2.1219 - accuracy: 0.2123 - val_loss: 2.7886 - val_accuracy: 0.0000e+00
Epoch 4/2000
31/31 [==============================] - 0s 11ms/step - loss: 1.9483 - accuracy: 0.3223 - val_loss: 3.3530 - val_accuracy: 0.0000e+00
Epoch 5/2000
31/31 [==============================] - 0s 11ms/step - loss: 1.7621 - accuracy: 0.3969 - val_loss: 3.3495 - val_accuracy: 0.0000e+00
Epoch 6/2000
31/31 [==============================] - 0s 11ms/step - loss: 1.5088 - accuracy: 0.4767 - val_loss: 3.5952 - val_accuracy: 0.0000e+00
Epoch 7

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/FNOW/MHEALTH.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/FNOW/MHEALTH.npz
(1335, 1, 250, 23)
Epoch 1/2000
16/16 [==============================] - 0s 25ms/step - loss: 2.2857 - accuracy: 0.1059 - val_loss: 2.8242 - val_accuracy: 0.0000e+00
Epoch 2/2000
16/16 [==============================] - 0s 12ms/step - loss: 2.2244 - accuracy: 0.1049 - val_loss: 2.8991 - val_accuracy: 0.0000e+00
Epoch 3/2000
16/16 [==============================] - 0s 11ms/step - loss: 2.1887 - accuracy: 0.1265 - val_loss: 2.8294 - val_accuracy: 0.0000e+00
Epoch 4/2000
16/16 [==============================] - 0s 11ms/step - loss: 2.1317 - accuracy: 0.1980 - val_loss: 2.8623 - val_accuracy: 0.0000e+00
Epoch 5/2000
16/16 [==============================] - 0s 12ms/step - loss: 2.0375 - accuracy: 0.2912 - val_loss: 3.0485 - val_accuracy: 0.0000e+00
Epoch 6/2000
16/16 [==============================] - 0s 12ms/step - loss: 1.8818 - accuracy: 0.3490 - val_loss: 3.2534 - val_accuracy: 0.0000e+00
Epoch 7

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOTO/MHEALTH.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOTO/MHEALTH.npz
(2555, 1, 250, 23)
Epoch 1/2000
31/31 [==============================] - 1s 17ms/step - loss: 2.3270 - accuracy: 0.1039 - val_loss: 2.6031 - val_accuracy: 0.0000e+00
Epoch 2/2000
31/31 [==============================] - 0s 11ms/step - loss: 2.2223 - accuracy: 0.1423 - val_loss: 2.6718 - val_accuracy: 0.0000e+00
Epoch 3/2000
31/31 [==============================] - 0s 11ms/step - loss: 2.0929 - accuracy: 0.2190 - val_loss: 2.8297 - val_accuracy: 0.0000e+00
Epoch 4/2000
31/31 [==============================] - 0s 11ms/step - loss: 1.8067 - accuracy: 0.3373 - val_loss: 2.9922 - val_accuracy: 0.0000e+00
Epoch 5/2000
31/31 [==============================] - 0s 11ms/step - loss: 1.5305 - accuracy: 0.4340 - val_loss: 2.9756 - val_accuracy: 0.0000e+00
Epoch 6/2000
31/31 [==============================] - 0s 11ms/step - loss: 1.3946 - accuracy: 0.4488 - val_loss: 3.0797 - val_accuracy: 0.0000e+00
Epoch 7

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/SNOW/MHEALTH.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/SNOW/MHEALTH.npz
(2555, 1, 250, 23)
Epoch 1/2000
31/31 [==============================] - 0s 16ms/step - loss: 2.3614 - accuracy: 0.1153 - val_loss: 2.6370 - val_accuracy: 0.0000e+00
Epoch 2/2000
31/31 [==============================] - 0s 11ms/step - loss: 2.2460 - accuracy: 0.1573 - val_loss: 2.7790 - val_accuracy: 0.0000e+00
Epoch 3/2000
31/31 [==============================] - 0s 11ms/step - loss: 2.0391 - accuracy: 0.2941 - val_loss: 3.1984 - val_accuracy: 0.0000e+00
Epoch 4/2000
31/31 [==============================] - 0s 11ms/step - loss: 1.7492 - accuracy: 0.3899 - val_loss: 3.1470 - val_accuracy: 0.0000e+00
Epoch 5/2000
31/31 [==============================] - 0s 11ms/step - loss: 1.4945 - accuracy: 0.4503 - val_loss: 3.4973 - val_accuracy: 0.0000e+00
Epoch 6/2000
31/31 [==============================] - 0s 11ms/step - loss: 1.2959 - accuracy: 0.5184 - val_loss: 4.2059 - val_accuracy: 0.0000e+00
Epoch 7

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOSO/USCHAD.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOSO/USCHAD.npz
(9824, 1, 500, 6)
Epoch 1/2000
119/119 [==============================] - 1s 8ms/step - loss: 2.1702 - accuracy: 0.2569 - val_loss: 2.7980 - val_accuracy: 0.0000e+00
Epoch 2/2000
119/119 [==============================] - 1s 6ms/step - loss: 1.6860 - accuracy: 0.3927 - val_loss: 3.9240 - val_accuracy: 0.0000e+00
Epoch 3/2000
119/119 [==============================] - 1s 6ms/step - loss: 1.4493 - accuracy: 0.4537 - val_loss: 5.2876 - val_accuracy: 0.0000e+00
Epoch 4/2000
119/119 [==============================] - 1s 6ms/step - loss: 1.3283 - accuracy: 0.4843 - val_loss: 5.9465 - val_accuracy: 0.0000e+00
Epoch 5/2000
119/119 [==============================] - 1s 6ms/step - loss: 1.2686 - accuracy: 0.5121 - val_loss: 6.3986 - val_accuracy: 0.0000e+00
Epoch 6/2000
119/119 [==============================] - 1s 6ms/step - loss: 1.2214 - accuracy: 0.5260 - val_loss: 6.6440 - val_accuracy: 0.0000e+00
Epo

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOTO/USCHAD.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOTO/USCHAD.npz
(9824, 1, 500, 6)
Epoch 1/2000
117/117 [==============================] - 1s 8ms/step - loss: 2.1215 - accuracy: 0.2688 - val_loss: 2.7790 - val_accuracy: 0.0000e+00
Epoch 2/2000
117/117 [==============================] - 1s 7ms/step - loss: 1.8080 - accuracy: 0.4095 - val_loss: 2.9590 - val_accuracy: 0.0000e+00
Epoch 3/2000
117/117 [==============================] - 1s 7ms/step - loss: 1.6088 - accuracy: 0.4682 - val_loss: 3.7639 - val_accuracy: 0.0000e+00
Epoch 4/2000
117/117 [==============================] - 1s 7ms/step - loss: 1.4090 - accuracy: 0.4948 - val_loss: 4.4433 - val_accuracy: 0.0000e+00
Epoch 5/2000
117/117 [==============================] - 1s 6ms/step - loss: 1.3125 - accuracy: 0.5144 - val_loss: 4.8997 - val_accuracy: 0.0000e+00
Epoch 6/2000
117/117 [==============================] - 1s 7ms/step - loss: 1.2638 - accuracy: 0.5350 - val_loss: 5.2368 - val_accuracy: 0.0000e+00
Epo

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/FNOW/USCHAD.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/FNOW/USCHAD.npz
(5038, 1, 500, 6)
Epoch 1/2000
61/61 [==============================] - 1s 9ms/step - loss: 2.2956 - accuracy: 0.1463 - val_loss: 2.6328 - val_accuracy: 0.0000e+00
Epoch 2/2000
61/61 [==============================] - 0s 7ms/step - loss: 2.0169 - accuracy: 0.2990 - val_loss: 2.7221 - val_accuracy: 0.0000e+00
Epoch 3/2000
61/61 [==============================] - 0s 7ms/step - loss: 1.8679 - accuracy: 0.3655 - val_loss: 2.8142 - val_accuracy: 0.0000e+00
Epoch 4/2000
61/61 [==============================] - 0s 7ms/step - loss: 1.6828 - accuracy: 0.4198 - val_loss: 3.1826 - val_accuracy: 0.0000e+00
Epoch 5/2000
61/61 [==============================] - 0s 7ms/step - loss: 1.4717 - accuracy: 0.4552 - val_loss: 3.8828 - val_accuracy: 0.0000e+00
Epoch 6/2000
61/61 [==============================] - 0s 7ms/step - loss: 1.3866 - accuracy: 0.4773 - val_loss: 4.0722 - val_accuracy: 0.0000e+00
Epoch 7/2000
61

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/SNOW/USCHAD.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/SNOW/USCHAD.npz
(9824, 1, 500, 6)
Epoch 1/2000
118/118 [==============================] - 1s 11ms/step - loss: 2.1588 - accuracy: 0.2582 - val_loss: 2.7209 - val_accuracy: 0.0000e+00
Epoch 2/2000
118/118 [==============================] - 1s 7ms/step - loss: 1.6648 - accuracy: 0.4165 - val_loss: 3.8032 - val_accuracy: 0.0000e+00
Epoch 3/2000
118/118 [==============================] - 1s 7ms/step - loss: 1.4148 - accuracy: 0.4614 - val_loss: 4.3769 - val_accuracy: 0.0000e+00
Epoch 4/2000
118/118 [==============================] - 1s 7ms/step - loss: 1.2908 - accuracy: 0.4884 - val_loss: 4.8138 - val_accuracy: 0.0000e+00
Epoch 5/2000
118/118 [==============================] - 1s 7ms/step - loss: 1.1637 - accuracy: 0.5266 - val_loss: 4.9338 - val_accuracy: 0.0023
Epoch 6/2000
118/118 [==============================] - 1s 7ms/step - loss: 1.0790 - accuracy: 0.5650 - val_loss: 5.5197 - val_accuracy: 0.0023
Epoch 7/20

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD1_1s.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD1_1s.npz
(3771, 1, 50, 6)
Epoch 1/2000
44/44 [==============================] - 0s 7ms/step - loss: 3.0237 - accuracy: 0.0559 - val_loss: 3.4769 - val_accuracy: 0.0000e+00
Epoch 2/2000
44/44 [==============================] - 0s 4ms/step - loss: 2.9707 - accuracy: 0.0633 - val_loss: 3.8111 - val_accuracy: 0.0000e+00
Epoch 3/2000
44/44 [==============================] - 1s 12ms/step - loss: 2.9251 - accuracy: 0.0761 - val_loss: 4.1408 - val_accuracy: 0.0000e+00
Epoch 4/2000
44/44 [==============================] - 0s 4ms/step - loss: 2.8565 - accuracy: 0.1081 - val_loss: 4.3669 - val_accuracy: 0.0000e+00
Epoch 5/2000
44/44 [==============================] - 0s 4ms/step - loss: 2.7845 - accuracy: 0.1131 - val_loss: 4.7656 - val_accuracy: 0.0000e+00
Epoch 6/2000
44/44 [==============================] - 0s 4ms/step - loss: 2.7414 - accuracy: 0.1113 - val_loss: 5.0417 - val_accuracy: 0.0000e+00
Epoch 7/2

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz
(3771, 1, 50, 6)
Epoch 1/2000
44/44 [==============================] - 0s 7ms/step - loss: 3.0390 - accuracy: 0.0567 - val_loss: 3.1113 - val_accuracy: 0.0000e+00
Epoch 2/2000
44/44 [==============================] - 0s 4ms/step - loss: 2.9963 - accuracy: 0.0649 - val_loss: 3.2293 - val_accuracy: 0.0000e+00
Epoch 3/2000
44/44 [==============================] - 0s 4ms/step - loss: 2.9405 - accuracy: 0.0707 - val_loss: 3.5804 - val_accuracy: 0.0000e+00
Epoch 4/2000
44/44 [==============================] - 0s 4ms/step - loss: 2.8450 - accuracy: 0.1080 - val_loss: 4.1970 - val_accuracy: 0.0000e+00
Epoch 5/2000
44/44 [==============================] - 0s 4ms/step - loss: 2.7799 - accuracy: 0.1115 - val_loss: 4.6179 - val_accuracy: 0.0000e+00
Epoch 6/2000
44/44 [==============================] - 0s 4ms/step - loss: 2.7149 - accuracy: 0.1298 - val_loss: 5.2157 - val_accuracy: 0.0000e+00
Epoch 7/20

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD1_1s.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD1_1s.npz
(2048, 1, 50, 6)
Epoch 1/2000
25/25 [==============================] - 0s 9ms/step - loss: 3.0249 - accuracy: 0.0635 - val_loss: 3.3759 - val_accuracy: 0.0000e+00
Epoch 2/2000
25/25 [==============================] - 0s 4ms/step - loss: 2.9838 - accuracy: 0.0757 - val_loss: 3.6395 - val_accuracy: 0.0000e+00
Epoch 3/2000
25/25 [==============================] - 0s 4ms/step - loss: 2.9501 - accuracy: 0.0731 - val_loss: 3.9411 - val_accuracy: 0.0000e+00
Epoch 4/2000
25/25 [==============================] - 0s 4ms/step - loss: 2.9243 - accuracy: 0.0738 - val_loss: 4.0151 - val_accuracy: 0.0000e+00
Epoch 5/2000
25/25 [==============================] - 0s 4ms/step - loss: 2.8788 - accuracy: 0.0770 - val_loss: 4.0536 - val_accuracy: 0.0000e+00
Epoch 6/2000
25/25 [==============================] - 0s 4ms/step - loss: 2.8388 - accuracy: 0.0956 - val_loss: 4.3946 - val_accuracy: 0.0000e+00
Epoch 7/20

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD1_1s.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD1_1s.npz
(3771, 1, 50, 6)
Epoch 1/2000
45/45 [==============================] - 0s 7ms/step - loss: 3.0003 - accuracy: 0.0639 - val_loss: 3.6442 - val_accuracy: 0.0000e+00
Epoch 2/2000
45/45 [==============================] - 0s 4ms/step - loss: 2.9378 - accuracy: 0.0761 - val_loss: 3.8807 - val_accuracy: 0.0118
Epoch 3/2000
45/45 [==============================] - 0s 4ms/step - loss: 2.8558 - accuracy: 0.1091 - val_loss: 4.1206 - val_accuracy: 0.0098
Epoch 4/2000
45/45 [==============================] - 0s 4ms/step - loss: 2.7727 - accuracy: 0.1147 - val_loss: 4.6099 - val_accuracy: 0.0118
Epoch 5/2000
45/45 [==============================] - 0s 4ms/step - loss: 2.7089 - accuracy: 0.1272 - val_loss: 4.8424 - val_accuracy: 0.0177
Epoch 6/2000
45/45 [==============================] - 0s 4ms/step - loss: 2.6803 - accuracy: 0.1275 - val_loss: 5.0956 - val_accuracy: 0.0315
Epoch 7/2000
45/45 [==========

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD2_1s.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD2_1s.npz
(1137, 1, 50, 6)
Epoch 1/2000
14/14 [==============================] - 0s 13ms/step - loss: 1.7448 - accuracy: 0.1837 - val_loss: 1.9967 - val_accuracy: 0.0000e+00
Epoch 2/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.6715 - accuracy: 0.2005 - val_loss: 2.1718 - val_accuracy: 0.0000e+00
Epoch 3/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.6410 - accuracy: 0.2125 - val_loss: 2.3010 - val_accuracy: 0.0000e+00
Epoch 4/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.5681 - accuracy: 0.2797 - val_loss: 2.5559 - val_accuracy: 0.0000e+00
Epoch 5/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.5260 - accuracy: 0.3205 - val_loss: 2.6397 - val_accuracy: 0.0000e+00
Epoch 6/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.4924 - accuracy: 0.3457 - val_loss: 2.8287 - val_accuracy: 0.0000e+00
Epoch 7/2

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz
(1137, 1, 50, 6)
Epoch 1/2000
14/14 [==============================] - 0s 13ms/step - loss: 1.7850 - accuracy: 0.1802 - val_loss: 1.8737 - val_accuracy: 0.0000e+00
Epoch 2/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.7234 - accuracy: 0.2148 - val_loss: 2.0782 - val_accuracy: 0.0000e+00
Epoch 3/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.6619 - accuracy: 0.2076 - val_loss: 2.1820 - val_accuracy: 0.0000e+00
Epoch 4/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.6194 - accuracy: 0.2255 - val_loss: 2.1517 - val_accuracy: 0.0000e+00
Epoch 5/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.5974 - accuracy: 0.2673 - val_loss: 2.1437 - val_accuracy: 0.0000e+00
Epoch 6/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.5359 - accuracy: 0.2876 - val_loss: 2.3378 - val_accuracy: 0.0000e+00
Epoch 7/2

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD2_1s.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD2_1s.npz
(616, 1, 50, 6)
Epoch 1/2000
8/8 [==============================] - 0s 20ms/step - loss: 1.7916 - accuracy: 0.1489 - val_loss: 1.8119 - val_accuracy: 0.0000e+00
Epoch 2/2000
8/8 [==============================] - 0s 7ms/step - loss: 1.7416 - accuracy: 0.1894 - val_loss: 1.8767 - val_accuracy: 0.0000e+00
Epoch 3/2000
8/8 [==============================] - 0s 7ms/step - loss: 1.7024 - accuracy: 0.2000 - val_loss: 1.9570 - val_accuracy: 0.0000e+00
Epoch 4/2000
8/8 [==============================] - 0s 6ms/step - loss: 1.6812 - accuracy: 0.1936 - val_loss: 2.0600 - val_accuracy: 0.0000e+00
Epoch 5/2000
8/8 [==============================] - 0s 7ms/step - loss: 1.6716 - accuracy: 0.1979 - val_loss: 2.1068 - val_accuracy: 0.0000e+00
Epoch 6/2000
8/8 [==============================] - 0s 6ms/step - loss: 1.6543 - accuracy: 0.2106 - val_loss: 2.2112 - val_accuracy: 0.0000e+00
Epoch 7/2000
8/8 [====

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD2_1s.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD2_1s.npz
(1137, 1, 50, 6)
Epoch 1/2000
14/14 [==============================] - 0s 12ms/step - loss: 1.7328 - accuracy: 0.1963 - val_loss: 1.9455 - val_accuracy: 0.0000e+00
Epoch 2/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.6772 - accuracy: 0.1963 - val_loss: 2.0520 - val_accuracy: 0.0000e+00
Epoch 3/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.6305 - accuracy: 0.2171 - val_loss: 2.1386 - val_accuracy: 0.0000e+00
Epoch 4/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.5837 - accuracy: 0.2540 - val_loss: 2.2152 - val_accuracy: 0.0000e+00
Epoch 5/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.5436 - accuracy: 0.3233 - val_loss: 2.2726 - val_accuracy: 0.0000e+00
Epoch 6/2000
14/14 [==============================] - 0s 5ms/step - loss: 1.4675 - accuracy: 0.3857 - val_loss: 2.5032 - val_accuracy: 0.0000e+00
Epoch 7/2

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOTO/WHARF.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOTO/WHARF.npz
(3871, 1, 160, 3)


ValueError: ignored

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz')

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/FNOW/WHARF.npz')

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/SNOW/WHARF.npz')

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOTO/WISDM.npz')

Kasnesis  et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOTO/WISDM.npz
(20846, 1, 100, 3)


ValueError: ignored

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOSO/WISDM.npz')

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/FNOW/WISDM.npz')

In [0]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/SNOW/WISDM.npz')